In [ ]:
import pandas as pd
import openai
import ipywidgets as widgets
from IPython.display import display, Image

# Load your dataset
df = pd.read_csv('Your_CSV_file.csv')

# Ensure comments and image URLs are valid (in case there are missing values)
df['comments'] = df['comments'].fillna("No comments available")
df['image_url'] = df['image_url'].fillna("")

# Step 1: Function to get all comments for a restaurant name
def get_restaurant_comments(restaurant_name):
    # Filter the DataFrame for the restaurant name
    restaurant = df[df['rest_name'].str.contains(restaurant_name, case=False, na=False)]
    
    if restaurant.empty:
        return None, None
    
    # Get the comments and image URL for the restaurant
    comments = restaurant['comments'].tolist()
    image_url = restaurant['image_url'].iloc[0]  # Assuming each restaurant has one image URL
    
    # Join all the comments into a single string
    all_comments = " ".join(comments)
    
    return all_comments, image_url

# Step 2: Function to summarize the comments using OpenAI API
def summarize_comments(restaurant_name):
    openai.api_key = 'Your api key'  # Replace with your actual OpenAI API key
    
    # Get the comments and image URL for the restaurant
    comments, image_url = get_restaurant_comments(restaurant_name)
    
    if comments is None:
        return "No restaurant found with that name.", None
    
    try:
        # Use OpenAI API to summarize comments
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # You can also use "gpt-4" if you have access
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Summarize the following restaurant reviews."},
                {"role": "user", "content": comments}
            ]
        )
        
        summary = response['choices'][0]['message']['content']
        return summary, image_url
    except Exception as e:
        return f"An error occurred while summarizing comments: {str(e)}", None

# Step 3: Function to display restaurant options and output
def display_restaurant_options():
    # Get a list of restaurant names
    restaurant_names = df['rest_name'].unique().tolist()

    # Create a dropdown menu with restaurant names
    dropdown = widgets.Dropdown(
        options=restaurant_names,
        description='Restaurants:',
        disabled=False,
    )

    # Output widget for showing the summary and image
    output = widgets.Output()

    def on_change(change):
        with output:
            output.clear_output()  # Clear previous output
            restaurant_name = change['new']
            summary, image_url = summarize_comments(restaurant_name)
            print(f"Summary for {restaurant_name}: {summary}")
            if image_url:
                display(Image(url=image_url))
            else:
                print("No image available for this restaurant.")

    # Trigger function when dropdown value changes
    dropdown.observe(on_change, names='value')

    # Display the dropdown and output widgets
    display(dropdown, output)

# Example Usage
display_restaurant_options()